In [1]:
import github_data_reader as reader
import chordpro_parser as parser

In [2]:
songs = reader.read_github_data('Asacri', 'asacriband-chords')
chunks = parser.parse_and_chunk(songs)

In [3]:
from qdrant_client import QdrantClient, models

In [4]:
qd_client = QdrantClient('http://localhost:6333')

In [5]:
dimensionality = 384
model_name = 'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2'
collection_name = "asacriband-chords"

In [6]:
if qd_client.collection_exists(collection_name):
    qd_client.delete_collection(collection_name)

qd_client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=dimensionality,
        distance=models.Distance.COSINE
    )
)

True

In [7]:
points = []

for i, doc in enumerate(chunks):
    text = doc.content
    text = text.strip()
    vector = models.Document(text=text, model=model_name)


    if (len(text) > 0):
        point = models.PointStruct(id=i, vector=vector, payload={ 'title': doc.title, 'content': doc.content, 'key': doc.key })
        points.append(point)

qd_client.upsert(
    collection_name=collection_name,
    points=points
)

/Users/kasteion/repos/kasteion/ai-bootcamp-project/.venv/lib/python3.13/site-packages/qdrant_client/embed/embedder.py:75: UserWarning: The model sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2 now uses mean pooling instead of CLS embedding. In order to preserve the previous behaviour, consider either pinning fastembed version to 0.5.1 or using `add_custom_model` functionality.
  model = TextEmbedding(model_name=model_name, **options)


UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [33]:
from pydantic_ai.messages import FunctionToolCallEvent

class NamedCallback:

    def __init__(self, agent):
        self.agent_name = agent.name

    async def print_function_calls(self, ctx, event):
        # Detect nested streams
        if hasattr(event, "__aiter__"):
            async for sub in event:
                await self.print_function_calls(ctx, sub)
            return

        if isinstance(event, FunctionToolCallEvent):
            tool_name = event.part.tool_name
            args = event.part.args
            print(f"TOOL CALL ({self.agent_name}): {tool_name}({args})")

    async def __call__(self, ctx, event):
        return await self.print_function_calls(ctx, event)


In [76]:
from typing import TypedDict, List

class SearchResult(TypedDict):
    """Represents a single search result entry."""
    title: str
    content: str
    key: str

class SearchTools:
    def vector_search(self, query: str, num_results=15) -> List[SearchResult]:
        """
        Performs a vector search looking for lyrics related with the query.

        Args:
            query (str): The search query string.

        Returns:
            List[SearchResult]: A list of search results. Each result dictionary contains:
                - title (str): The title of the song.
                - content (str): The content of the song that matched.
                - key (str): The key the of the song.
        """
        vector = models.Document(text=query, model=model_name)

        query_points = qd_client.query_points(
            collection_name=collection_name,
            query=vector,
            limit=num_results,
            with_payload=True
        )

        results = []

        for point in query_points.points:
            results.append(point.payload)

        return results

    def get_full_song_by_title(self, title: str) -> SearchResult:
        """
        Retrieve full song by title

        Args:
            title (str): The title of the song
        
        Returns:
            SearchResult: The full song
                - title (str): The title of the song.
                - content (str): The content of the song.
                - key (str): The key the of the song.
        """
        sections = [chunk for chunk in chunks if chunk.title == title]
        if len(sections) == 0:
            return ""
        
        title = sections[0].title
        key = sections[0].key
        content = ' '.join([section.content for section in sections])
        return SearchResult(title = title, content=content, key=key)


In [77]:
tools = SearchTools()

In [57]:
# import re
# section_content = re.sub(r"\[[^\]]+\]", "", songs[1].content)
# print(section_content)
print(tools.get_full_song_by_title("2da de Corintios 5:17"))


{'title': '2da de Corintios 5:17', 'content': 'El que está en Cristo es nueva criatura. Las cosas viejas pasaron ya. He aquí todas son hechas nuevas, Así se inicia el primer amor. Por Cristo vivimos, en Cristo morimos, En Él redimimos el mal que hicimos. Su sangre nos limpia de todo pecado Y así el pasado queda sepultado.', 'key': 'G'}


In [58]:
' '.join([chunk.content for chunk in chunks if chunk.title == 'En Totalidad a Ti'])

'En totalidad a Ti, Rindo alma y corazón, No tengo nada más que dar Tómalo mi buen Señor Tómame en tus brazos Cristo, Toma mi vida en tus manos Y haz de mi lo que Tú quieras, Tómame aquí estoy Tómame aquí estoy'

In [80]:
from pydantic_ai import Agent

instructions = """
Eres un agente cuyo objetivo es recomendar un listado de canciones 
basadas en un tema en el que el usuario se quiera enfocar.

OBJETIVO

Dado el tema en que el usuario quiere enfocarse, ejecuta búsquedas 
en la base de datos con la letra de las canciones para encontra canciones
relacionadas con el tema. Encuentra 4 o 5 canciones diferentes para usar 
como repertorio de canciones basadas en ese tema.

PROCESO

Fase 1 - Búsqueda inicial

- Tomando el input del usuario crea 5 palabras claves diferentes relacionadas con.
- Utiliza vector_search() para buscar en la base de datos canciones que puedan estar relacionadas con ese tema.

Fase 2 - Generar lista

- Genera una lista de 4 a 5 títulos de las canciones

Fase 3 - Evaluar lista

- Con el listado de la Fase 2 
- Utiliza `get_full_song_by_title()` para obtener la canción completa
- Evalúa cada una de las canciones contra el input del usuario para asignarle un score de 0 a 100


FORMATO DEL OUTPUT

Al final, escribe tu recomendación siguiendo esta estructura.

**Entrada** <input original>

**Títulos recomendados**
- Bullet titles, keys y score de las canciones

**Punteo del repertorio**
Analisis final del repertorio y un punteo general

REGLAS

- Usa siempre `vector_search()` para buscar canciones por su letras
- Usa `get_full_song_by_title()` para obtener la canción completa
- Haz siempre al menos 5 búsquedas únicas
- Recomienda siempre títulos diferentes
- No recomiendes más de 5 títulos
"""

agent_tools = [tools.vector_search, tools.get_full_song_by_title]

agent = Agent(
    name="recomendations",
    instructions=instructions,
    tools=agent_tools,
    model='gpt-4o-mini'
)

In [81]:
results = await agent.run(
    user_prompt='yo me rindo a él',
    event_stream_handler=NamedCallback(agent)
)

TOOL CALL (recomendations): vector_search({"query":"yo me rindo a él"})
TOOL CALL (recomendations): get_full_song_by_title({"title": "Al Rey de reyes - A Ti sea la gloria"})
TOOL CALL (recomendations): get_full_song_by_title({"title": "Creo en Ti"})
TOOL CALL (recomendations): get_full_song_by_title({"title": "Santo"})
TOOL CALL (recomendations): get_full_song_by_title({"title": "Gracias Cristo"})
TOOL CALL (recomendations): get_full_song_by_title({"title": "Aquel que nos amó"})


In [82]:
print(results.output)

**Entrada** yo me rindo a él

**Títulos recomendados**
- **Al Rey de reyes - A Ti sea la gloria** (Key: A) - Score: 95
- **Creo en Ti** (Key: A) - Score: 90
- **Santo** (Key: D) - Score: 85
- **Gracias Cristo** (Key: A) - Score: 80
- **Aquel que nos amó** (Key: G) - Score: 88

**Punteo del repertorio**
Este repertorio está centrado en la adoración y entrega a un ser divino. Las canciones reflejan un profundo sentido de rendición y alabanza, capturando la esencia de "yo me rindo a él". La mayor puntuación la tiene "Al Rey de reyes - A Ti sea la gloria", que directamente habla de ofrecer adoración. "Creo en Ti" también resuena fuertemente con el tema de fe y rendición. En general, todas las canciones recomendadas están alineadas con el espíritu de entrega y adoración. El repertorio es equilibrado, con letras que inspiran y elevan el alma.
